In [37]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction import DictVectorizer
from scipy.stats import ks_2samp

### 01 Data Import

In [4]:
# Specify data types (as given in assignment)

column_types = {
    'uuid': 'object',
    'default': 'float64',
    'account_amount_added_12_24m': 'float64',
    'account_days_in_dc_12_24m': 'float64',
    'account_days_in_rem_12_24m': 'float64',
    'account_days_in_term_12_24m': 'float64',
    'account_incoming_debt_vs_paid_0_24m': 'float64',
    'account_status': 'category',
    'account_worst_status_0_3m': 'category',
    'account_worst_status_12_24m': 'category',
    'account_worst_status_3_6m': 'category',
    'account_worst_status_6_12m': 'category',
    'age': 'int64',
    'avg_payment_span_0_12m': 'float64',
    'avg_payment_span_0_3m': 'float64',
    'merchant_category': 'category',
    'merchant_group': 'category',
    'has_paid': 'bool',
    'max_paid_inv_0_12m': 'float64',
    'max_paid_inv_0_24m': 'float64',
    'name_in_email': 'category',
    'num_active_div_by_paid_inv_0_12m': 'float64',
    'num_active_inv': 'float64',
    'num_arch_dc_0_12m': 'float64',
    'num_arch_dc_12_24m': 'float64',
    'num_arch_ok_0_12m': 'float64',
    'num_arch_ok_12_24m': 'float64',
    'num_arch_rem_0_12m': 'float64',
    'num_arch_written_off_0_12m': 'float64',
    'num_arch_written_off_12_24m': 'float64',
    'num_unpaid_bills': 'float64',
    'status_last_archived_0_24m': 'category',
    'status_2nd_last_archived_0_24m': 'category',
    'status_3rd_last_archived_0_24m': 'category',
    'status_max_archived_0_6_months': 'category',
    'status_max_archived_0_12_months': 'category',
    'status_max_archived_0_24_months': 'category',
    'recovery_debt': 'float64',
    'sum_capital_paid_account_0_12m': 'float64',
    'sum_capital_paid_account_12_24m': 'float64',
    'sum_paid_inv_0_12m': 'float64',
    'time_hours': 'float64',
    'worst_status_active_inv': 'category'
}


# Import data
df = pd.read_csv("dataset.csv", delimiter=';', dtype=column_types)

In [5]:
# Take a first look at the data
print(df.shape)

(99976, 43)


In [6]:
df.head()

,uuid,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,account_status,account_worst_status_0_3m,account_worst_status_12_24m,...,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours,worst_status_active_inv
0,63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0.0,0.0,0.0,0.0,0.0,0.0,1,1,NaN,...,1,1,1,1,0.0,0.0,0.0,178839.0,9.653333,1
1,0e961183-8c15-4470-9a5e-07a1bd207661,0.0,0.0,0.0,0.0,0.0,NaN,1,1,1,...,1,1,2,2,0.0,0.0,0.0,49014.0,13.181389,NaN
2,d8edaae6-4368-44e0-941e-8328f203e64e,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1,1,2,2,0.0,0.0,0.0,124839.0,11.561944,1
3,0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,0.0,0.0,0.0,324676.0,15.751111,1
4,c8f8b835-5647-4506-bf15-49105d8af30b,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,0,1,1,1,0.0,0.0,0.0,7100.0,12.698611,NaN


### 02 EDA + Preprocessing

In [7]:
# Take a look at missing values
missing_values = df.isna().sum()
missing_values_percentage = (df.isnull().sum() / len(df)) * 100

missing_info = pd.DataFrame({'Missing Count': missing_values, 'Missing Percentage': missing_values_percentage})
missing_info = missing_info.sort_values(by='Missing Count', ascending=False)

missing_info

,Missing Count,Missing Percentage
worst_status_active_inv,69515,69.531688
account_worst_status_12_24m,66761,66.777026
account_worst_status_6_12m,60350,60.364487
account_incoming_debt_vs_paid_0_24m,59315,59.329239
account_worst_status_3_6m,57702,57.715852
account_status,54373,54.386053
account_worst_status_0_3m,54373,54.386053
avg_payment_span_0_3m,49305,49.316836
avg_payment_span_0_12m,23836,23.841722
num_active_div_by_paid_inv_0_12m,22939,22.944507


In [8]:
# Check out the distribution of target variable and features
df.default.value_counts()

## Highly imbalanced

default
0.0    88688
1.0     1288
Name: count, dtype: int64

In [9]:
# Check for outliers in numerical features
pd.set_option('display.float_format', lambda x: '%.2f' % x)
# Get numeric columns
df_numeric = df.select_dtypes(include='number').copy()
df_numeric.describe(percentiles=[.05,.25, .5, .75,.95,.98,.99])


,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,age,avg_payment_span_0_12m,avg_payment_span_0_3m,max_paid_inv_0_12m,...,num_arch_ok_12_24m,num_arch_rem_0_12m,num_arch_written_off_0_12m,num_arch_written_off_12_24m,num_unpaid_bills,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours
count,89976.00,99976.00,88140.00,88140.00,88140.00,40661.00,99976.00,76140.00,50671.00,99976.00,...,99976.00,99976.00,81898.00,81898.00,99976.00,99976.00,99976.00,99976.00,99976.00,99976.00
mean,0.01,12255.15,0.22,5.04,0.29,1.33,36.02,17.97,14.99,9203.65,...,6.37,0.47,0.00,0.00,2.14,4.04,10816.07,6542.90,39208.80,15.33
std,0.12,35481.48,5.81,22.86,2.93,26.48,13.00,12.75,10.30,13512.17,...,15.35,1.36,0.01,0.01,6.30,163.93,26463.97,19041.22,90649.29,5.03
min,0.00,0.00,0.00,0.00,0.00,0.00,18.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5%,0.00,0.00,0.00,0.00,0.00,0.00,19.00,5.14,3.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7.36
25%,0.00,0.00,0.00,0.00,0.00,0.00,25.00,10.80,8.40,2000.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2600.00,11.62
50%,0.00,0.00,0.00,0.00,0.00,0.15,34.00,14.91,13.00,6052.00,...,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,15995.00,15.79
75%,0.00,4937.25,0.00,0.00,0.00,0.66,45.00,21.00,18.29,11380.00,...,6.00,0.00,0.00,0.00,2.00,0.00,9029.75,85.00,43844.25,19.54
95%,0.00,72967.00,0.00,31.00,0.00,2.79,60.00,41.00,36.00,29272.50,...,28.00,2.00,0.00,0.00,10.00,0.00,57993.75,39200.75,150288.50,22.34
98%,0.00,121295.00,0.00,76.00,0.00,7.56,67.00,53.67,47.00,45600.00,...,50.00,4.00,0.00,0.00,18.00,0.00,92899.00,67499.00,240852.00,23.09


In [10]:
# Check for data integrity issues:
 # account_days_in_rem (dc)_12_24m has a max value of 365. I would assume the max would be 730 (24 months). It's worth checking how the variable was created in order to confirm there's not an error somewhere.
 # Other quick checks - age is capped at 100, time is capped at 24 hours, monitary values seem reasonable (there are no values in the billions/trillions)
# What is the difference between NA and 0 in variables like account_days_in_dc_12_24m

In [11]:
# There are outliers in the data. I would like to cap the numerical features at the 98th percentile, because XGBoost can focus too much on outliers.
# But first there are some features that are sparse but strong signals. I work with these seperately to not lose those features.
pd.crosstab(pd.cut(df['account_days_in_dc_12_24m'], [-1,0,365]), df['default'], normalize='index')

default,0.00,1.00
account_days_in_dc_12_24m,,
"(-1, 0]",0.99,0.01
"(0, 365]",0.81,0.19


In [12]:
# Calculate the percentage of zeros for each numerical feature
zero_percentage = (df_numeric == 0).sum() / df_numeric.notna().sum()
#print(zero_percentage)
# Identify features where more than 98% of the values are zeros
sparse_features = zero_percentage[zero_percentage > .98]

print(sparse_features)

sparse_features_list = sparse_features.index.tolist()


default                       0.99
account_days_in_dc_12_24m     1.00
account_days_in_term_12_24m   0.99
num_arch_written_off_0_12m    1.00
num_arch_written_off_12_24m   1.00
recovery_debt                 1.00
dtype: float64


In [13]:
sparse_features_list

['default',
 'account_days_in_dc_12_24m',
 'account_days_in_term_12_24m',
 'num_arch_written_off_0_12m',
 'num_arch_written_off_12_24m',
 'recovery_debt']

In [14]:
# turn these into boolean features

for feature in sparse_features_list:
    df.loc[df[feature] > 1, feature] = 1

In [15]:
df.account_days_in_dc_12_24m.value_counts()

account_days_in_dc_12_24m
0.00    87879
1.00      261
Name: count, dtype: int64

In [16]:
df_numeric_cols = df_numeric.columns.tolist()
df_numeric_cols_filtered = [col for col in df_numeric_cols if col not in sparse_features_list]
df_numeric_cols_filtered

['account_amount_added_12_24m',
 'account_days_in_rem_12_24m',
 'account_incoming_debt_vs_paid_0_24m',
 'age',
 'avg_payment_span_0_12m',
 'avg_payment_span_0_3m',
 'max_paid_inv_0_12m',
 'max_paid_inv_0_24m',
 'num_active_div_by_paid_inv_0_12m',
 'num_active_inv',
 'num_arch_dc_0_12m',
 'num_arch_dc_12_24m',
 'num_arch_ok_0_12m',
 'num_arch_ok_12_24m',
 'num_arch_rem_0_12m',
 'num_unpaid_bills',
 'sum_capital_paid_account_0_12m',
 'sum_capital_paid_account_12_24m',
 'sum_paid_inv_0_12m',
 'time_hours']

In [17]:
# exclude these feature, and cap the rest at 98 percentile
df_numeric_cols = df_numeric.columns.tolist()
df_numeric_cols_filtered = [col for col in df_numeric_cols if col not in sparse_features_list]

percentile_98 = df_numeric.quantile(0.98)

# Cap the outliers in each numerical column to the 98th percentile value
for feature in df_numeric_cols_filtered:
    cap_value = percentile_98[feature]
    df[feature] = df[feature].clip(upper=cap_value)

# Confirm capping worked
df[df_numeric.columns].describe(percentiles=[.05,.25, .5, .75,.95,.98,.99])

,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,age,avg_payment_span_0_12m,avg_payment_span_0_3m,max_paid_inv_0_12m,...,num_arch_ok_12_24m,num_arch_rem_0_12m,num_arch_written_off_0_12m,num_arch_written_off_12_24m,num_unpaid_bills,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours
count,89976.00,99976.00,88140.00,88140.00,88140.00,40661.00,99976.00,76140.00,50671.00,99976.00,...,99976.00,99976.00,81898.00,81898.00,99976.00,99976.00,99976.00,99976.00,99976.00,99976.00
mean,0.01,10811.11,0.00,3.84,0.01,0.63,35.92,17.56,14.83,8583.93,...,5.61,0.41,0.00,0.00,1.82,0.00,9827.85,5762.19,35213.78,15.32
std,0.12,25707.60,0.05,14.04,0.12,1.33,12.72,10.69,9.67,9594.44,...,10.06,0.89,0.01,0.01,3.57,0.05,20518.21,14156.93,50989.84,5.02
min,0.00,0.00,0.00,0.00,0.00,0.00,18.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5%,0.00,0.00,0.00,0.00,0.00,0.00,19.00,5.14,3.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7.36
25%,0.00,0.00,0.00,0.00,0.00,0.00,25.00,10.80,8.40,2000.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2600.00,11.62
50%,0.00,0.00,0.00,0.00,0.00,0.15,34.00,14.91,13.00,6052.00,...,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,15995.00,15.79
75%,0.00,4937.25,0.00,0.00,0.00,0.66,45.00,21.00,18.29,11380.00,...,6.00,0.00,0.00,0.00,2.00,0.00,9029.75,85.00,43844.25,19.54
95%,0.00,72967.00,0.00,31.00,0.00,2.79,60.00,41.00,36.00,29272.50,...,28.00,2.00,0.00,0.00,10.00,0.00,57993.75,39200.75,150288.50,22.34
98%,0.00,121295.00,0.00,76.00,0.00,7.56,67.00,53.67,47.00,45600.00,...,50.00,4.00,0.00,0.00,18.00,0.00,92898.50,67499.00,240852.00,23.09


In [18]:
## Categorical features
df_cat = df.select_dtypes(include='category').copy()


In [19]:
# Check how many levels the features have
df_cat.nunique()

account_status                      4
account_worst_status_0_3m           4
account_worst_status_12_24m         4
account_worst_status_3_6m           4
account_worst_status_6_12m          4
merchant_category                  57
merchant_group                     12
name_in_email                       8
status_last_archived_0_24m          5
status_2nd_last_archived_0_24m      5
status_3rd_last_archived_0_24m      5
status_max_archived_0_6_months      4
status_max_archived_0_12_months     5
status_max_archived_0_24_months     5
worst_status_active_inv             3
dtype: int64

In [20]:
df_cat.merchant_category.value_counts()

merchant_category
Diversified entertainment                                  38614
Youthful Shoes & Clothing                                  11755
Books & Magazines                                           9363
General Shoes & Clothing                                    4597
Concept stores & Miscellaneous                              4406
Sports gear & Outdoor                                       3712
Dietary supplements                                         3101
Diversified children products                               2994
Diversified electronics                                     1844
Prints & Photos                                             1675
Children Clothes & Nurturing products                       1500
Pet supplies                                                1315
Electronic equipment & Related accessories                  1037
Jewelry & Watches                                            911
Hobby articles                                               910
Prescri

In [21]:
# Group rare instances into 'Other' category and then one hot encode

# Define a threshold for rare categories. Categories with fewer instances than this threshold will be grouped into 'Other'
threshold = 100

# Count the frequency of each category in the feature
freq_count = df.merchant_category.value_counts()
# Find rare categories that are below the threshold
rare_categories = freq_count[freq_count < threshold].index
# Replace rare categories with 'Other'
df['merchant_category'] = df['merchant_category'].apply(lambda x: 'Other' if x in rare_categories else x)

# Perform one-hot encoding
#df_one_hot = pd.get_dummies(df, columns=df_cat.columns)

# Display the shape and first few rows of the one-hot encoded DataFrame
#df_one_hot.shape


(99976, 140)

In [ ]:
# One hot encoding with a dict vectorizor is better in production 
# for treating categories that are not present in the training data



In [38]:
df_one_hot.head()

,uuid,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,age,avg_payment_span_0_12m,avg_payment_span_0_3m,...,status_max_archived_0_12_months_3,status_max_archived_0_12_months_5,status_max_archived_0_24_months_0,status_max_archived_0_24_months_1,status_max_archived_0_24_months_2,status_max_archived_0_24_months_3,status_max_archived_0_24_months_5,worst_status_active_inv_1,worst_status_active_inv_2,worst_status_active_inv_3
0,63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0.00,0.00,0.00,0.00,0.00,0.00,20,12.69,8.33,...,False,False,False,True,False,False,False,True,False,False
1,0e961183-8c15-4470-9a5e-07a1bd207661,0.00,0.00,0.00,0.00,0.00,NaN,50,25.83,25.00,...,False,False,False,False,True,False,False,False,False,False
2,d8edaae6-4368-44e0-941e-8328f203e64e,0.00,0.00,0.00,0.00,0.00,NaN,22,20.00,18.00,...,False,False,False,False,True,False,False,True,False,False
3,0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0.00,0.00,NaN,NaN,NaN,NaN,36,4.69,4.89,...,False,False,False,True,False,False,False,True,False,False
4,c8f8b835-5647-4506-bf15-49105d8af30b,0.00,0.00,0.00,0.00,0.00,NaN,25,13.00,13.00,...,False,False,False,True,False,False,False,False,False,False


In [22]:
rare_categories

CategoricalIndex(['Kitchenware',
                  'Household electronics (whitegoods/appliances)', 'Underwear',
                  'Erotic Clothing & Accessories', 'Non',
                  'Musical Instruments & Equipment', 'Tobacco',
                  'Safety products', 'Diversified Jewelry & Accessories',
                  'Car electronics', 'Sex toys', 'Plants & Flowers',
                  'Bags & Wallets',
                  'Office machines & Related accessories (excl. computers)',
                  'Cleaning & Sanitary', 'Event tickets',
                  'Wine, Beer & Liquor', 'Education'],
                 categories=['Adult Shoes & Clothing', 'Automotive Parts & Accessories', 'Bags & Wallets', 'Body & Hair Care', ..., 'Wine, Beer & Liquor', 'Youthful Shoes & Clothing', 'Office machines & Related accessories (excl. ..., 'Education'], ordered=False, dtype='category', name='merchant_category')

In [23]:
df.merchant_category.nunique()

40

In [24]:
# Examine categorical variables crossed with default
for feature in df.select_dtypes(include='category').columns:
    print(pd.crosstab(feature, df['default'], normalize='index'))

default         0.00  1.00
row_0                     
account_status  0.99  0.01
default                    0.00  1.00
row_0                                
account_worst_status_0_3m  0.99  0.01
default                      0.00  1.00
row_0                                  
account_worst_status_12_24m  0.99  0.01
default                    0.00  1.00
row_0                                
account_worst_status_3_6m  0.99  0.01
default                     0.00  1.00
row_0                                 
account_worst_status_6_12m  0.99  0.01
default         0.00  1.00
row_0                     
merchant_group  0.99  0.01
default        0.00  1.00
row_0                    
name_in_email  0.99  0.01
default                     0.00  1.00
row_0                                 
status_last_archived_0_24m  0.99  0.01
default                         0.00  1.00
row_0                                     
status_2nd_last_archived_0_24m  0.99  0.01
default                         0.00  1.00
row_0 

### 03 Modeling

In [25]:
# Split the dataset into training and validation sets
train = df_one_hot[df_one_hot.default.notna()]
validate = df_one_hot[df_one_hot.default.isna()]

features = train.drop(columns=['uuid', 'default'])
target = train['default']

# Weight ratio for class imbalance
weight_ratio = float(len(target[target == 1]))/float(len(target[target == 0]))
w_array = np.array([1.0]*target.shape[0])
w_array[target==1] = 1 - weight_ratio
w_array[target==0] = weight_ratio

# Initialize the XGBoost model
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.1,
    'random_state': 42
}

dtrain = xgb.DMatrix(features, label=target, weight=w_array)
#cv_results = xgb.cv(params, dtrain, nfold=5, num_boost_round=300, early_stopping_rounds=50, maximize=True, seed=42)
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=300,
    nfold=5,
    metrics=['auc'],
    early_stopping_rounds=50,
    stratified=True,
    verbose_eval=True
)


# Extract the best AUC and KS from the CV results
best_auc = cv_results['test-auc-mean'].max()

print(f"Best ROC AUC from CV: {best_auc}")


[0]	train-auc:0.89119+0.00398	test-auc:0.85534+0.01072
[1]	train-auc:0.89990+0.00320	test-auc:0.86774+0.01445
[2]	train-auc:0.90596+0.00203	test-auc:0.87209+0.01549
[3]	train-auc:0.90995+0.00088	test-auc:0.87543+0.01518
[4]	train-auc:0.91413+0.00044	test-auc:0.87852+0.01374
[5]	train-auc:0.91635+0.00104	test-auc:0.88024+0.01346
[6]	train-auc:0.91838+0.00131	test-auc:0.88204+0.01245
[7]	train-auc:0.91986+0.00166	test-auc:0.88386+0.01145
[8]	train-auc:0.92191+0.00164	test-auc:0.88536+0.01116
[9]	train-auc:0.92377+0.00154	test-auc:0.88715+0.01133
[10]	train-auc:0.92564+0.00165	test-auc:0.88846+0.01071
[11]	train-auc:0.92715+0.00128	test-auc:0.89007+0.01037
[12]	train-auc:0.92878+0.00130	test-auc:0.89103+0.01006
[13]	train-auc:0.93008+0.00091	test-auc:0.89161+0.00982
[14]	train-auc:0.93137+0.00114	test-auc:0.89188+0.00918
[15]	train-auc:0.93255+0.00095	test-auc:0.89322+0.00882
[16]	train-auc:0.93362+0.00087	test-auc:0.89391+0.00863
[17]	train-auc:0.93446+0.00081	test-auc:0.89451+0.00858
[1

In [26]:
final_model = xgb.train(params, dtrain, num_boost_round=cv_results.shape[0])

#final_model.save_model('final_model.model')


In [30]:
import pickle

In [31]:
model_num = 1.0
output_file = f'model_i={model_num}.bin'
output_file

'model_i=1.0.bin'

In [36]:
# wb -> write binary (bytes, not text)
f_out = open(output_file, 'wb')
pickle.dump((dv, final_model), f_out)
f_out.close()

NameError: name 'dv' is not defined

In [ ]:
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)
    # do stuff

# file is closed automatically    

In [34]:
# Prepare the validation data
validation_features = validate.drop(columns=['uuid', 'default'])
dval = xgb.DMatrix(validation_features)

# Make predictions on the validation set
y_pred = final_model.predict(dval)

# Create a DataFrame to hold the uuid and predicted probabilities
result_df = pd.DataFrame({
    'uuid': validate['uuid'],
    'pd': y_pred
})

# Save the result to a CSV file
result_df.to_csv('predicted_default_probabilities.csv', index=False)


In [35]:
pred_proba

array([0.083715  , 0.14744486, 0.05339191, ..., 0.9470487 , 0.06943693,
       0.15819046], dtype=float32)